## 0. Set up

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np
import urllib.request
import tensorflow as tf
import transformers
import re
import os
from tqdm import tqdm
from tensorflow import keras

## 1. Load Dataset

In [8]:
dataset = pd.read_excel('/content/drive/MyDrive/sentiment_sentences_changed.xlsx', names=['sentence', 'sentiment', 'sub_sentiment'])

In [5]:
dataset.sample(n=10)

,sentence,sentiment,sub_sentiment
121201,아빠에게 나를 믿어달라고 말할 거야.,슬픔,낙담한
2808,뿌듯하고 기쁜 마음을 오래 기억하고 싶어.,기쁨,기쁨
10036,매장 내부도 다시 보고 재료를 확인할 거야. 청결하게 관리해야지.,기쁨,신이 난
50915,아무래도 가족들과 대화를 좀 해 봐야겠어. 왜 그러는지.,분노,분노
23186,맞아. 날 갑자기 날 괴롭히기 시작해서 괴로워.,당황,당황
13736,저녁 먹고 아내랑 산책하는 게 요즘 사는 낙이야.,기쁨,편안한
121352,맛있는 저녁을 먹고 바람 좀 쐬려고 해.,슬픔,낙담한
100356,조금씩 배웠어도 이 지경까지 되진 않았을 텐데 너무 후회돼.,상처,괴로워하는
104532,얼마 전에 암 수술을 했어. 몸이 안 좋아서 일할 수가 없으니 생활이 너무 어렵네.,상처,버려진
104353,나 오래 연애하던 여자친구와 헤어졌어. 버려진 기분이 들어서 슬퍼.,상처,버려진


## 2. Preprocess Dataset

In [6]:
# 결측값 및 중복 샘플 제거
def drop_na_and_duplicates(df, col):
  df = df.dropna(how='any')
  df = df.drop_duplicates(subset=[col])
  df = df.reset_index(drop=True)
  return df

In [7]:
dataset = drop_na_and_duplicates(dataset, dataset.columns[0])

In [8]:
print(f'총 데이터 개수: {len(dataset)}')

총 데이터 개수: 144723


In [9]:
sub_sentiment_cnt = dataset['sub_sentiment'].nunique()
print(f'감정 라벨 개수: {sub_sentiment_cnt}')

감정 라벨 개수: 58


In [10]:
dataset.groupby(by=['sub_sentiment']).count()

,sentence,sentiment
sub_sentiment,,
"가난한, 불우한",2698,2698
감사하는,1720,1720
걱정스러운,3461,3461
고립된,5083,5083
괴로워하는,2677,2677
구역질 나는,2463,2463
기쁨,1819,1819
낙담한,2529,2529
남의 시선을 의식하는,2488,2488


In [9]:
# labeling

# label 58개 전체 사용 -> labeling.xlsx 사용 / left_on='sub_sentiment'
# label 6개만 사용 -> labeling2.xlsx 사용 / left_on='sentiment'
labels = pd.read_excel('/content/drive/MyDrive/labeling.xlsx')
dataset = pd.merge(dataset, labels, left_on='sub_sentiment', right_on='sentiment')
dataset = dataset[['sentence', 'label']]

In [12]:
dataset.sample(n=10)

,sentence,label
102603,난 병원 신세라 언제 떠날지 모르는데 빚과 함께 남을 아들을 생각하니 안타까워.,18
25168,일도 못하고 이렇게 병원신세만 지다 죽을까 두려워. 모든 게 후회스러워.,13
24550,오늘 딸이 내 발에 걸려 넘어지는 바람에 울어서 놀랐어.,13
45440,코딩 개념 자체를 아무리 설명을 들어도 머리만 아프고 이해를 못하겠어.,52
46737,지저분하고 더러워서 못 살겠어.,5
75263,그래서 요즘 근래 모임도 같이 나가고 자주 보기는 해.,2
24646,남편이 같이 점심 먹자고 직장으로 찾아온 적은 처음이라 당황스러워.,13
36100,그래야지. 먹는 양 줄이고 운동 시작할 거야. 처방약도 꾸준히 복용해야지.,42
140257,내가 돈이 없으니 자식들도 내 말을 듣지 않고 나를 괄시하는 게 느껴져서 환멸이 나.,53
109584,어머니 십 주기 추도식에 다녀왔어. 벌써 시간이 그렇게나 흘렀네.,34


In [13]:
max_seq_len = 0
len_64_128 = 0
len_128_256 = 0
mylist = [0 for i in range(160)]
for index, value in dataset['sentence'].items():
  mylist[len(value)] += 1
  max_seq_len = max(max_seq_len, len(value))
  if len(value) >= 64 and len(value) < 128:
    len_64_128 += 1
  elif len(value) >= 128 and len(value) < 256:
    len_128_256 += 1

print(f'가장 긴 문장 길이: {max_seq_len}')
print(f'64과 128 사이 길이 문장 개수: {len_64_128}')
print(f'128과 256 사이 길이 문장 개수: {len_128_256}')

가장 긴 문장 길이: 156
64과 128 사이 길이 문장 개수: 2374
128과 256 사이 길이 문장 개수: 3


In [14]:
max_seq_len = 128

In [10]:
from sklearn.model_selection import train_test_split

dataset_X = dataset[dataset.columns[0]].tolist()
dataset_y = dataset[dataset.columns[1]].tolist()

# train/validation/test dataset 분할
# train:val:test = 6:2:2
train_X, temp_X, train_y, temp_y = train_test_split(dataset_X, dataset_y, test_size=0.4, random_state=0)
val_X, test_X, val_y, test_y = train_test_split(temp_X, temp_y, test_size=0.5, random_state=0)

In [11]:
print(f'train set length: {len(train_X)}')
print(f'valid set length: {len(val_X)}')
print(f'test  set length: {len(test_X)}')

train set length: 86833
valid set length: 28945
test  set length: 28945


## 3. Tokenize Dataset

In [13]:
from transformers import BertTokenizerFast

In [18]:
tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base", max_len=max_seq_len, truncation=True, padding=True) # klue/bert

In [19]:
train_X = tokenizer(train_X, truncation=True, padding=True)
val_X = tokenizer(val_X, truncation=True, padding=True)

In [20]:
# tensorflow의 dataset 객체로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_X), train_y))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_X), val_y))

## 4. Fine-tuning Model

In [12]:
from transformers import TFBertForSequenceClassification

In [22]:
model_name = "klue/bert-base"
num_labels = len(dataset.groupby(by=['label']))
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

In [23]:
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, from_pt=True)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
callback_earlystop = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=0
)

In [25]:
train_dataset = train_dataset.shuffle(10000)
train_dataset = train_dataset.batch(64)
val_dataset = val_dataset.shuffle(10000)
val_dataset = val_dataset.batch(64)

In [26]:
model.fit(
    train_dataset, epochs=3, batch_size=64,
    validation_data = val_dataset,
    callbacks = [callback_earlystop]
)

Epoch 1/3
1357/1357 [==============================] - 264s 157ms/step - loss: 3.4731 - accuracy: 0.1651 - val_loss: 3.2163 - val_accuracy: 0.2168
Epoch 2/3
1357/1357 [==============================] - 194s 143ms/step - loss: 3.0988 - accuracy: 0.2361 - val_loss: 3.1422 - val_accuracy: 0.2283
Epoch 3/3
1357/1357 [==============================] - 193s 142ms/step - loss: 2.8891 - accuracy: 0.2742 - val_loss: 3.1452 - val_accuracy: 0.2327


## 5. Save Model

In [28]:
MODEL_NAME = 'best_bert'
MODEL_SAVE_PATH = os.path.join("/content/drive/MyDrive", MODEL_NAME)

if os.path.exists(MODEL_SAVE_PATH):
  print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
  os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
  print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/drive/MyDrive/best_bert -- Folder create complete 



('/content/drive/MyDrive/best_bert/tokenizer_config.json',
 '/content/drive/MyDrive/best_bert/special_tokens_map.json',
 '/content/drive/MyDrive/best_bert/vocab.txt',
 '/content/drive/MyDrive/best_bert/added_tokens.json',
 '/content/drive/MyDrive/best_bert/tokenizer.json')

## 6. Load Model

In [14]:
from transformers import TextClassificationPipeline

# Load Fine-tuned model
MODEL_NAME = 'best_bert'
MODEL_SAVE_PATH = os.path.join("/content/drive/MyDrive", MODEL_NAME)

loaded_tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    return_all_scores=True
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/best_bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


## 7. Prediction Model

In [15]:
# label 58개 전체 사용 -> labeling.xlsx 사용
# label 6개만 사용 -> labeling2.xlsx 사용
df = pd.read_excel('/content/drive/MyDrive/labeling.xlsx')

In [16]:
def sentiment_predict(sentence):
  preds_list = text_classifier(sentence)[0]

  sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
  predicted_label = int(re.sub(r'[^0-9]', '', sorted_preds_list[0]['label']))
  predicted_score = sorted_preds_list[0]['score']
  real_label = df[df['label'] == predicted_label]['sentiment'].values

  print(f'{predicted_score:.2%} 확률로 {predicted_label}, {real_label}에 속하는 감정 문장입니다.')

In [17]:
def sentiment_all_predict(sentence):
  preds_list = text_classifier(sentence)[0]

  pos_sum = 0
  sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
  for i in range(0, len(sorted_preds_list)):
    predicted_label = int(re.sub(r'[^0-9]', '', sorted_preds_list[i]['label']))
    predicted_score = sorted_preds_list[i]['score']
    real_label = df[df['label'] == predicted_label]['sentiment'].values
    pos_sum += predicted_score

    print(f'{predicted_score:.2%} 확률로 {real_label}에 속하는 감정 문장입니다.')

In [18]:
test_set = pd.DataFrame({'sentence': test_X, 'label': test_y})

In [19]:
predicted_label_list = []
predicted_score_list = []

for text in tqdm(test_set['sentence']):
  preds_list = text_classifier(text)[0]

  sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
  predicted_label_list.append(int(re.sub(r'[^0-9]', '', sorted_preds_list[0]['label'])))
  predicted_score_list.append(sorted_preds_list[0]['score'])

100%|██████████| 28945/28945 [3:28:20<00:00,  2.32it/s]


In [24]:
test_set['pred'] = predicted_label_list
test_set['score'] = predicted_score_list
test_set.sample(n=10)

,sentence,label,pred,score
27333,어학 성적도 있어야 하고 관련 분야 자격증들도 있어야지. 그런데 자격증들이 너무 많...,25,25,0.174657
5846,솔직하게 나의 단점을 미리 말해주는 것도 하나의 방법이겠지.,14,47,0.167723
10558,며칠 전에 무릎이 아파서 정형외과에 갔더니 퇴행성 관절염이라네.,52,52,0.060358
16909,응. 줄곧 그러지 말라고 얘기했었는데 이제는 스트레스받을 지경이야.,26,26,0.418709
19060,부모님이 돌아가시고 고생하고 있을 형제들을 생각하니 슬퍼.,38,10,0.254380
14528,내 체력에서 가능한 운동을 할 거야. 조금씩 하다 보면 건강도 좋아질 거야.,15,35,0.046843
7343,평소에도 나를 본체만체하는데 앞으로는 더 무시하겠지.,36,17,0.190723
14550,응. 이번 결혼기념일에 남편이랑 해외여행 가기로 했거든. 이십 주년 기념으로 가는데...,29,29,0.599084
23698,맞아 부모님께서 지금까지 나를 도와주시지 않았다면 난 취업에 성공하지 못 했을 거야.,1,1,0.535330
25178,우울증으로 인해 흥미도 없고 뭘 하든 힘들어.,26,40,0.586288


In [21]:
test_set.groupby(by=['pred']).count()

,sentence,label,score
pred,,,
0,659,659,659
1,497,497,497
2,1046,1046,1046
3,1517,1517,1517
4,396,396,396
5,374,374,374
6,423,423,423
7,318,318,318
8,480,480,480


## 8. Evaluate Model

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_true=test_set['label'], y_pred=test_set['pred']))

              precision    recall  f1-score   support

           0       0.22      0.27      0.24       528
           1       0.37      0.52      0.43       352
           2       0.16      0.26      0.20       646
           3       0.19      0.30      0.24       979
           4       0.25      0.18      0.21       534
           5       0.26      0.20      0.23       500
           6       0.21      0.23      0.22       383
           7       0.21      0.12      0.15       536
           8       0.25      0.26      0.26       459
           9       0.12      0.09      0.11       548
          10       0.28      0.25      0.26       552
          11       0.24      0.31      0.27       298
          12       0.43      0.18      0.25       491
          13       0.28      0.18      0.22       461
          14       0.23      0.24      0.24       599
          15       0.14      0.13      0.14       490
          16       0.29      0.23      0.26       350
          17       0.16    